<a href="https://colab.research.google.com/github/diem-ai/topic-modeling/blob/master/Topic_Modeling_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Introduction

#### Every document we read can be thought of as consisting of many topics all stacked upon one another. Today, we’re going can unpack these topics using of NLP techniques: 
- Latent Dirichlet Allocation (LDA) and Topic Modeling
- Data is collected on https://www.reuters.com/breakingviews by a scrapping script
- The goal is to break text documents down into topics by word. 
- What is laten feature ? Mathematically, we want to find “topics” that are collections of words that appear in similar documents. 
  More generally, it is a collection of features in a dataset.
- There are several libraries for LDA such as scikit-learn and gensim. I choose gensim for this project. 

#### Project tasks:
- Cleaning the dataset & Lemmatization
- Creat a dictionay from processed data
- Create Corpus and LDA Model with bag of words
- Create Coprpus and LDA with TF-IDF
- Caculate the Perplexity and Topic Cohenrence between two models
- Visualize topics with the help of pyLDAvis


#### Google Colab Setup

In [22]:
from google.colab import drive
# This will prompt for authorization.
# authorization code: 4/OwErfUj6QceGXhIGx_RWv0MKclb9rilw8UsJnZqFbSez-QS8zQ399JU
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
!pip install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [0]:
#import accessory_functions.py from Colab
#https://drive.google.com/open?id=1S7URZIBq4zMh5QWv0qXPHv4ixhgHWN_y

my_module = drive.CreateFile({'id':'1S7URZIBq4zMh5QWv0qXPHv4ixhgHWN_y'})
my_module.GetContentFile('accessory_functions.py')

In [25]:
!pip install unidecode

In [26]:
!pip install pyLDAvis

<p>Data Path & Model parameters</p>

In [0]:
datapath = '/content/drive/My Drive/data/'
n_topics = 50
iterations = 50

<p> Import Libraries </p>

In [0]:
import numpy as np
import string
import pandas as pd
import unidecode

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from accessory_functions import read_pickle_file


"""


import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
nltk.download('averaged_perceptron_tagger')
"""

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt
%matplotlib inline
# Make all my plots 538 Style
plt.style.use('fivethirtyeight')

import warnings
warnings.simplefilter('ignore')


<p> Loading data</p>

In [0]:
processed_docs = read_pickle_file(datapath + 'processed_docs.pkl')
bow = read_pickle_file(datapath + 'bow.pkl')
tfidf = read_pickle_file(datapath + 'tfidf.pkl')
dictionary = read_pickle_file(datapath + 'dictionary.pkl')

In [30]:
print(processed_docs[1])



['trade', 'flow', 'traditional', 'economic', 'measure', 'reveal', 'true', 'cost', 'tariff', 'washington', 'friday', 'hike', 'duty', 'billion', 'chinese', 'good', 'side', 'intimate', 'survive', 'blow', 'may', 'slightly', 'dent', 'gdp', 'shift', 'supply', 'chain', 'though', 'show', 'extent', 'long', 'term', 'loss']


In [31]:
doc = bow[1]

print(doc)

[(dictionary[id], count) for id, count in doc]

[(4, 1), (10, 1), (33, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1)]


[('billion', 1),
 ('friday', 1),
 ('trade', 1),
 ('blow', 1),
 ('chain', 1),
 ('chinese', 1),
 ('cost', 1),
 ('dent', 1),
 ('duty', 1),
 ('economic', 1),
 ('extent', 1),
 ('flow', 1),
 ('gdp', 1),
 ('good', 1),
 ('hike', 1),
 ('intimate', 1),
 ('long', 1),
 ('loss', 1),
 ('may', 1),
 ('measure', 1),
 ('reveal', 1),
 ('shift', 1),
 ('show', 1),
 ('side', 1),
 ('slightly', 1),
 ('supply', 1),
 ('survive', 1),
 ('tariff', 1),
 ('term', 1),
 ('though', 1),
 ('traditional', 1),
 ('true', 1),
 ('washington', 1)]

In [32]:
doc = tfidf[1]

print(doc)

[(dictionary[id], freq) for id, freq in doc]

[(4, 0.05775843375562911), (10, 0.12310661992926017), (33, 0.10439212162929286), (38, 0.1961303786255056), (39, 0.17310433437273764), (40, 0.10148900465133513), (41, 0.1199009983425769), (42, 0.23725627027918395), (43, 0.21614153934185534), (44, 0.1071217734433688), (45, 0.28633219743654287), (46, 0.1961303786255056), (47, 0.1961303786255056), (48, 0.1091949636509986), (49, 0.1923966831387611), (50, 0.28633219743654287), (51, 0.11962217757715381), (52, 0.1524456522665324), (53, 0.06964385147961394), (54, 0.17420790459371643), (55, 0.19813974994500413), (56, 0.1765129312952131), (57, 0.12161692894792232), (58, 0.1829493640280444), (59, 0.21010281715361415), (60, 0.18157469132600734), (61, 0.21010281715361415), (62, 0.16023546606560787), (63, 0.13550592942070178), (64, 0.11798968502713748), (65, 0.19813974994500413), (66, 0.20487183805983358), (67, 0.14595088124716785)]


[('billion', 0.05775843375562911),
 ('friday', 0.12310661992926017),
 ('trade', 0.10439212162929286),
 ('blow', 0.1961303786255056),
 ('chain', 0.17310433437273764),
 ('chinese', 0.10148900465133513),
 ('cost', 0.1199009983425769),
 ('dent', 0.23725627027918395),
 ('duty', 0.21614153934185534),
 ('economic', 0.1071217734433688),
 ('extent', 0.28633219743654287),
 ('flow', 0.1961303786255056),
 ('gdp', 0.1961303786255056),
 ('good', 0.1091949636509986),
 ('hike', 0.1923966831387611),
 ('intimate', 0.28633219743654287),
 ('long', 0.11962217757715381),
 ('loss', 0.1524456522665324),
 ('may', 0.06964385147961394),
 ('measure', 0.17420790459371643),
 ('reveal', 0.19813974994500413),
 ('shift', 0.1765129312952131),
 ('show', 0.12161692894792232),
 ('side', 0.1829493640280444),
 ('slightly', 0.21010281715361415),
 ('supply', 0.18157469132600734),
 ('survive', 0.21010281715361415),
 ('tariff', 0.16023546606560787),
 ('term', 0.13550592942070178),
 ('though', 0.11798968502713748),
 ('traditiona

<p>Build LDA Model with Bag-of-Word and Calculate Perplexity</p>

In [0]:
lda_bow = gensim.models.LdaModel(bow      
                                 , num_topics=n_topics
                                       , id2word=dictionary
                                       , iterations=iterations)

<p>Print top 10 popular topics</p>

In [34]:
topics = lda_bow.print_topics(10)

for idx, topic in topics:
  print("topic: {}\n {}".format(idx, topic))
  
#[print("topic: {}\n {}".format(idx, topic)) for idx, topic in topics]

topic: 1
 0.009*"billion" + 0.008*"inc" + 0.007*"job" + 0.007*"trln" + 0.006*"election" + 0.006*"make" + 0.006*"prove" + 0.006*"news" + 0.005*"presidential" + 0.005*"interview"
topic: 26
 0.015*"billion" + 0.007*"investor" + 0.007*"tv" + 0.007*"company" + 0.006*"vote" + 0.006*"year" + 0.006*"though" + 0.005*"shareholder" + 0.005*"could" + 0.005*"new"
topic: 20
 0.010*"billion" + 0.009*"president" + 0.008*"state" + 0.007*"trump" + 0.007*"still" + 0.006*"like" + 0.006*"deal" + 0.006*"china" + 0.005*"investment" + 0.005*"trade"
topic: 29
 0.014*"bank" + 0.008*"bond" + 0.008*"could" + 0.008*"president" + 0.007*"largely" + 0.007*"euro" + 0.006*"european" + 0.006*"billion" + 0.006*"criminal" + 0.006*"price"
topic: 39
 0.010*"company" + 0.010*"billion" + 0.007*"apple" + 0.006*"year" + 0.006*"cash" + 0.006*"capitol" + 0.005*"one" + 0.005*"executive" + 0.005*"mid" + 0.005*"go"
topic: 3
 0.008*"market" + 0.007*"elect" + 0.006*"create" + 0.006*"new" + 0.006*"chamber" + 0.006*"chinese" + 0.006*"es

In [35]:
# Compute Perplexity
# a measure of how good the model is. the lower, the better.
print('\nPerplexity: ', lda_bow.log_perplexity(bow)) 

# Compute Coherence Score
coherence_lda_bow = CoherenceModel(model=lda_bow
                                     , corpus=bow
                                     , texts = processed_docs
                                     , dictionary=dictionary 
                                     ,coherence='c_v')

coherence_lda = coherence_lda_bow.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.695302707640202

Coherence Score:  0.3247636975491756


<p>Visualize the topics-keywords</p>

In [36]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_bow, bow, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
12    -0.034826  0.063385       1        1  4.462560
47    -0.053256  0.001113       2        1  4.089017
31    -0.088283  0.006707       3        1  3.770911
28    -0.021713 -0.019402       4        1  3.718895
48    -0.124392 -0.028702       5        1  3.411261
27     0.007027  0.033399       6        1  2.878317
32     0.002165  0.076224       7        1  2.827271
26     0.025466  0.088789       8        1  2.823118
0     -0.117754  0.019056       9        1  2.753980
18     0.008658  0.043237      10        1  2.660764
49    -0.085709 -0.025293      11        1  2.584152
40     0.045078  0.028210      12        1  2.331280
6     -0.103795  0.000029      13        1  2.311081
20    -0.011448  0.018250      14        1  2.245770
14     0.057560  0.122846      15        1  2.235251
23     0.010735 -0.013313      16        1  2.166156
21    -0.036620 -0.006885      17        1  2.155827
39     0.009182  0.084425      18        1  2.125574
35     0.017058  0.025444      19        1  2.107124
16    -0.045121 -0.048288      20        1  2.097232
41    -0.037061  0.010126      21        1  2.074632
13     0.012997  0.061211      22        1  2.051269
43    -0.069155 -0.021302      23        1  2.048844
2     -0.016792  0.033707      24        1  2.000197
44    -0.018353  0.036037      25        1  1.970255
5     -0.068337 -0.003507      26        1  1.926075
8     -0.108681 -0.045150      27        1  1.842120
46    -0.061545  0.004897      28        1  1.697498
4     -0.006052  0.016382      29        1  1.690452
11     0.078395  0.040059      30        1  1.627249
45     0.037930 -0.055617      31        1  1.621048
42     0.076695  0.088191      32        1  1.620156
38     0.015518 -0.036139      33        1  1.553123
3      0.036065 -0.046768      34        1  1.482350
25    -0.035873 -0.047889      35        1  1.462330
29     0.041011 -0.011448      36        1  1.443233
19     0.010159 -0.020890      37        1  1.404526
15     0.044177  0.019613      38        1  1.401396
10    -0.016722 -0.009767      39        1  1.314804
36     0.068532 -0.001641      40        1  1.290462
34    -0.009337 -0.060322      41        1  1.284403
33     0.006954 -0.048187      42        1  1.250283
17     0.075910  0.009327      43        1  1.192598
7      0.056650  0.011554      44        1  1.160949
22     0.053885 -0.044587      45        1  1.110994
30     0.043773 -0.144523      46        1  1.108633
37     0.053616 -0.037259      47        1  1.095059
1      0.040489 -0.001412      48        1  1.094014
24     0.076966 -0.079909      49        1  0.904348
9      0.158174 -0.084019      50        1  0.521162, topic_info=      Category        Freq            Term       Total  loglift  logprob
416    Default  797.000000           trump  797.000000  30.0000  30.0000
148    Default  719.000000       president  719.000000  29.0000  29.0000
402    Default  604.000000          donald  604.000000  28.0000  28.0000
167    Default  394.000000            bank  394.000000  27.0000  27.0000
4146   Default   98.000000          senate   98.000000  26.0000  26.0000
700    Default  124.000000           court  124.000000  25.0000  25.0000
244    Default  406.000000           could  406.000000  24.0000  24.0000
2151   Default   72.000000           elect   72.000000  23.0000  23.0000
3401   Default   77.000000         supreme   77.000000  22.0000  22.0000
1172   Default   90.000000    presidential   90.000000  21.0000  21.0000
6357   Default   40.000000       kavanaugh   40.000000  20.0000  20.0000
299    Default  412.000000             say  412.000000  19.0000  19.0000
467    Default  258.000000       executive  258.000000  18.0000  18.0000
506    Default  191.000000           house  191.000000  17.0000  17.0000
6355   Default   39.000000           brett   39.000000  16.0000  16.0000
936    Default  108.000000       

#### LDA model with TF-IDF


In [0]:
lda_tfidf = gensim.models.LdaModel(tfidf
                                       , num_topics=n_topics
                                       , id2word=dictionary
                                       , iterations=iterations)

<p>Print top 10 topics</p>

In [38]:
#pprint(lda_model.print_topics())
for idx, topic in lda_tfidf.print_topics(5):
    print('\nTopic: {}\nWords: {}'.format(idx, topic))



Topic: 2
Words: 0.009*"supreme" + 0.008*"court" + 0.007*"sexual" + 0.006*"federal" + 0.005*"allegation" + 0.005*"dec" + 0.005*"oversee" + 0.004*"assault" + 0.004*"governor" + 0.004*"antonio"

Topic: 14
Words: 0.007*"lawsuit" + 0.006*"court" + 0.005*"jia" + 0.005*"supreme" + 0.004*"security" + 0.004*"authority" + 0.004*"violate" + 0.004*"cry" + 0.004*"female" + 0.003*"season"

Topic: 45
Words: 0.004*"jeff" + 0.004*"fraud" + 0.004*"proceed" + 0.004*"money" + 0.004*"approach" + 0.003*"online" + 0.003*"question" + 0.003*"package" + 0.003*"answer" + 0.003*"information"

Topic: 21
Words: 0.006*"ethic" + 0.005*"accept" + 0.004*"automaker" + 0.004*"accuse" + 0.004*"indian" + 0.004*"truth" + 0.004*"boy" + 0.003*"senator" + 0.003*"ellison" + 0.003*"deal"

Topic: 23
Words: 0.006*"presidential" + 0.005*"goldman" + 0.005*"russian" + 0.004*"bond" + 0.004*"lloyd" + 0.004*"sachs" + 0.004*"nbc" + 0.004*"election" + 0.004*"interference" + 0.004*"morgan"


#### Compute Perplexity & Coherence Score

In [39]:
# Compute Perplexity
# a measure of how good the model is. the lower, the better.
print('\nPerplexity: ', lda_tfidf.log_perplexity(tfidf)) 

# Compute Coherence Score
coherence_model_tfidf = CoherenceModel(model=lda_tfidf
                                     , corpus=tfidf
                                     , texts = processed_docs
                                     , dictionary=dictionary 
                                     ,coherence='c_v')

coherence_tfidf = coherence_model_tfidf.get_coherence()
print('\nCoherence Score: ', coherence_tfidf)



Perplexity:  -17.246296709852626

Coherence Score:  0.5328383044511587


#### Visualize the topics

In [40]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_tfidf, tfidf, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
41    -0.099090  0.033613       1        1  5.264269
6     -0.051787  0.096983       2        1  4.887484
22    -0.073888  0.010357       3        1  4.497945
43    -0.069229  0.076149       4        1  4.205158
45    -0.040987  0.053093       5        1  4.020063
13    -0.089148 -0.055172       6        1  3.580301
1     -0.073732  0.015036       7        1  3.440197
4     -0.099666 -0.138970       8        1  3.203334
11    -0.067922  0.010230       9        1  2.988463
32    -0.060337 -0.033674      10        1  2.734752
21    -0.020723  0.032595      11        1  2.632566
8     -0.025469  0.043708      12        1  2.605289
19    -0.009909 -0.033707      13        1  2.244062
36    -0.037119  0.002989      14        1  2.181559
10    -0.013995 -0.019056      15        1  2.083648
33     0.000481 -0.000845      16        1  2.041333
34     0.024024  0.026190      17        1  1.983168
48     0.004444 -0.001771      18        1  1.959926
3      0.003526  0.004342      19        1  1.876661
25     0.021361  0.014995      20        1  1.801481
18     0.007483  0.004935      21        1  1.773840
39     0.011705 -0.012435      22        1  1.756099
46    -0.005525 -0.058501      23        1  1.671130
44    -0.009059  0.003705      24        1  1.616528
31     0.020263  0.024088      25        1  1.594631
2      0.019747 -0.029944      26        1  1.566739
24     0.004747 -0.000934      27        1  1.534676
42     0.032307  0.009318      28        1  1.517170
30     0.008004 -0.039810      29        1  1.498440
17     0.042859 -0.001671      30        1  1.478118
7      0.016396  0.009592      31        1  1.438293
28     0.019468 -0.012395      32        1  1.404542
23     0.030571 -0.011305      33        1  1.392936
38     0.020072 -0.006364      34        1  1.382710
49     0.011242  0.009307      35        1  1.371160
0      0.021514  0.010410      36        1  1.369874
14     0.025514 -0.001752      37        1  1.334913
35     0.032990 -0.000690      38        1  1.328905
9      0.032716  0.011317      39        1  1.246597
16     0.033575  0.008533      40        1  1.227770
5      0.033547 -0.005539      41        1  1.192266
47     0.038373 -0.001397      42        1  1.189356
27     0.016963  0.000549      43        1  1.160806
26     0.037747  0.008930      44        1  1.153988
20     0.048770 -0.012181      45        1  1.030412
40     0.050896 -0.001736      46        1  0.958373
15     0.041989 -0.005395      47        1  0.944727
29     0.041362 -0.009215      48        1  0.921263
12     0.047387 -0.023354      49        1  0.885131
37     0.045545 -0.003148      50        1  0.826955, topic_info=      Category       Freq            Term      Total  loglift  logprob
854    Default   9.000000         twitter   9.000000  30.0000  30.0000
616    Default  11.000000            post  11.000000  29.0000  29.0000
240    Default   9.000000       statement   9.000000  28.0000  28.0000
11091  Default   7.000000        verified   7.000000  27.0000  27.0000
700    Default  13.000000           court  13.000000  26.0000  26.0000
3401   Default   8.000000         supreme   8.000000  25.0000  25.0000
4007   Default  10.000000         account  10.000000  24.0000  24.0000
4146   Default  11.000000          senate  11.000000  23.0000  23.0000
629    Default  11.000000          follow  11.000000  22.0000  22.0000
1560   Default   8.000000        personal   8.000000  21.0000  21.0000
6357   Default   7.000000       kavanaugh   7.000000  20.0000  20.0000
6355   Default   7.000000           brett   7.000000  19.0000  19.0000
6562   Default   6.000000       immigrant   6.000000  18.0000  18.0000
3215   Default   8.000000           judge   8.000000  17.0000  17.0000
2894   Default   8.000000       secretary   8.000000  16.0000  16.0000
416    Default  40.000000           trump  40.000000  15.0000  15

<p>Test model with unseen data</p>

In [41]:
text = 'Uber Technologies lackluster stock-market debut is a warning for other tech unicorns'

unseen_doc = dictionary.doc2bow(text.split())

vector = lda_bow[unseen_doc]

for idx, score in vector:
  print("topic: {} score: {}".format(idx, lda_bow.print_topic(idx)))



topic: 10 score: 0.015*"president" + 0.014*"fargo" + 0.011*"kavanaugh" + 0.011*"donald" + 0.010*"trump" + 0.010*"brett" + 0.008*"investment" + 0.008*"fake" + 0.008*"murder" + 0.007*"executive"
topic: 24 score: 0.017*"senate" + 0.009*"year" + 0.008*"leader" + 0.008*"could" + 0.007*"government" + 0.007*"bill" + 0.007*"party" + 0.006*"capitalist" + 0.006*"may" + 0.005*"european"
topic: 27 score: 0.009*"bank" + 0.008*"financial" + 0.007*"china" + 0.007*"could" + 0.007*"year" + 0.006*"may" + 0.006*"billion" + 0.005*"come" + 0.005*"well" + 0.005*"make"
topic: 44 score: 0.008*"president" + 0.008*"presidential" + 0.007*"counsel" + 0.007*"special" + 0.006*"china" + 0.006*"mueller" + 0.006*"market" + 0.006*"group" + 0.006*"make" + 0.006*"election"


In [42]:
text = 'Uber Technologies lackluster stock-market debut is a warning for other tech unicorns'

unseen_doc = dictionary.doc2bow(text.split())

vector = lda_tfidf[unseen_doc]

for idx, score in vector:
  print("topic: {} score: {}".format(idx, lda_tfidf.print_topic(idx)))


topic: 9 score: 0.006*"abortion" + 0.005*"agency" + 0.005*"northern" + 0.004*"ireland" + 0.004*"strict" + 0.004*"de" + 0.003*"tank" + 0.003*"facto" + 0.003*"necessary" + 0.003*"make"
topic: 15 score: 0.009*"tower" + 0.005*"kalanick" + 0.005*"travis" + 0.004*"toll" + 0.004*"mexican" + 0.004*"serve" + 0.004*"shift" + 0.003*"korea" + 0.003*"reduce" + 0.003*"possibly"
topic: 42 score: 0.007*"le" + 0.005*"pyongyang" + 0.005*"la" + 0.004*"hour" + 0.004*"operation" + 0.004*"store" + 0.004*"hinge" + 0.004*"propaganda" + 0.004*"alibaba" + 0.003*"queue"
topic: 46 score: 0.006*"merkel" + 0.006*"attorney" + 0.006*"department" + 0.005*"germany" + 0.004*"mueller" + 0.004*"robert" + 0.004*"counsel" + 0.004*"confused" + 0.004*"justice" + 0.004*"angela"


<p>With the same text, both models give the different results</p>